
### Assignment 3 Submission - Asset Pricing
> Student Name : Ankit Rawat

> Batch : G2 

In [30]:
import pandas as pd
import random
import numpy as np
import pandas as pd
import pandas_datareader.data as web
import datetime
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import subplot, scatter, plot, axis
from scipy.stats import linregress
import ast 
import re
import seaborn as sns
import plotly.figure_factory as ff
import plotly.graph_objs as go
import scipy.stats as stats
import numpy as np
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm

pd.set_option("display.notebook_repr_html", False)
pd.set_option("display.max_columns", 12)
pd.set_option("display.width", 80)
pd.set_option("precision", 3)

In [31]:
%matplotlib inline
pd.set_option("max_rows", 180)
pd.set_option("min_rows", 70)

### 1. Load Risk Factors


In [32]:
df_risk_factors = pd.read_csv("Risk_Factors.csv")
df_market_Portfolio = pd.read_csv("Market_Portfolio.csv")
data_indus_Portfolio  = pd.read_csv("Industry_Portfolios.csv")

df_risk_factors.head(10)
df_risk_factors.set_index("Date",inplace = True)
df_market_Portfolio.set_index("Date",inplace = True)
data_indus_Portfolio.set_index("Date",inplace = True)

e = pd.DataFrame([1] * len( data_indus_Portfolio))

In [19]:
df_ = pd.concat([data_indus_Portfolio,df_market_Portfolio,df_risk_factors],axis =1)
df_1  = df_.copy()
df_2 = np.subtract(df_.loc[:,"NoDur":"Market"], pd.DataFrame(df_["Rf"]))

### 1.Treynor Ratio

In [20]:
Treynor_ratio = []
for i in df_.loc[:,"NoDur":"Other"].columns:
     Beta = (df_[i ].cov(df_1["Market"]))/np.square(df_1["Market"].std()).mean()
     ratio = df_2[i].mean()/Beta
     Treynor_ratio.append((i,ratio))
    
Treynor_ratio = pd.DataFrame(Treynor_ratio,columns = {"Sector","Treynor_Ratio"})
Treynor_ratio

  Treynor_Ratio  Sector
0         NoDur   1.188
1         Durbl   0.368
2         Manuf   0.757
3         Enrgy   1.138
4         HiTec   0.564
5         Telcm   0.837
6         Shops   0.955
7          Hlth   0.976
8         Utils   1.449
9         Other   0.300

## 2. Sharpe Ratio 

In [21]:
Sharpe_ratio1  = []
for i in df_.loc[:,"NoDur":"Other"].columns:
     M = df_[i ].mean()
     ratio = df_2[i].mean()/df_1[i].std()
     Sharpe_ratio1.append((ratio,i))
    
Sharpe_ratio = pd.DataFrame(Sharpe_ratio1,columns = {"Sector","Sharpe_Ratio"})
Sharpe_ratio

   Sharpe_Ratio Sector
0         0.232  NoDur
1         0.072  Durbl
2         0.167  Manuf
3         0.182  Enrgy
4         0.119  HiTec
5         0.170  Telcm
6         0.193  Shops
7         0.173   Hlth
8         0.211  Utils
9         0.065  Other

### 3. Alpha

In [22]:
Alpha_ratio = []
for i in df_.loc[:,"NoDur":"Other"].columns:
     Beta = (df_[i ].cov(df_1["Market"]))/np.square(df_1["Market"].std()).mean()
     difference = df_2[i].mean() - df_2["Market"].mean() * Beta
     Alpha_ratio.append((i,difference))
    
Alpha_ratio = pd.DataFrame(Alpha_ratio,columns = {"Sector","Alpha_ratio"})
Alpha_ratio

  Alpha_ratio  Sector
0       NoDur   0.370
1       Durbl  -0.417
2       Manuf   0.159
3       Enrgy   0.502
4       HiTec  -0.064
5       Telcm   0.195
6       Shops   0.276
7        Hlth   0.239
8       Utils   0.446
9       Other  -0.388

### 4. Sortino Ratio 

In [23]:
Sortino_ratio = []
Sortino_ratio1 = []
for i in df_.loc[:,"NoDur":"Other"].columns:
     Downside_risk2 = df_2[i].apply( lambda x : 0 if x > 0 else x )
     Downside_risk_std = Downside_risk2.std()
     ratio = df_2[i].mean() / Downside_risk_std
     Sortino_ratio.append((ratio,i))

Sortino_ratio = pd.DataFrame(Sortino_ratio,columns = {"Sortino_ratio","Sector"})
Sortino_ratio

   Sortino_ratio Sector
0          0.386  NoDur
1          0.126  Durbl
2          0.264  Manuf
3          0.306  Enrgy
4          0.195  HiTec
5          0.274  Telcm
6          0.327  Shops
7          0.310   Hlth
8          0.315  Utils
9          0.097  Other

### 5.Factor_ratio

In [24]:
Factor_ratio = []
Rm_Rf = df_["Rm-Rf"]
for i in df_.loc[:,"NoDur":"Other"].columns:
    placeholder2 = df_.loc[:,i] - df_["Rf"]
    model = sm.formula.ols(formula = "placeholder2 ~ Rm_Rf +SMB+HML", data = df_).fit()
    Factor_ratio.append([i,model.params[0]])



Factor_ratio = pd.DataFrame(Factor_ratio, columns = {"Sector","3FA"})
Factor_ratio

  Sector    3FA
0  NoDur  0.387
1  Durbl -0.474
2  Manuf  0.153
3  Enrgy  0.523
4  HiTec -0.066
5  Telcm  0.201
6  Shops  0.256
7   Hlth  0.257
8  Utils  0.474
9  Other -0.404

### 6 . Performance Matrix 

In [25]:
Performance_metrics = pd.concat( [Treynor_ratio.set_index("Sector"),Factor_ratio.set_index("Sector"),Sortino_ratio.set_index("Sector"),Alpha_ratio.set_index("Sector"),Sharpe_ratio.set_index("Sector")],axis =1 )
Performance_metrics

                    Treynor_Ratio    3FA  Sortino_ratio Alpha_ratio  \
Sector                                                                
-0.4173825493542932           NaN    NaN            NaN       Durbl   
-0.3877046222235476           NaN    NaN            NaN       Other   
-0.0643851934675187           NaN    NaN            NaN       HiTec   
0.15930375764243              NaN    NaN            NaN       Manuf   
0.19496392337002422           NaN    NaN            NaN       Telcm   
0.23874033896123237           NaN    NaN            NaN        Hlth   
0.2759693007229006            NaN    NaN            NaN       Shops   
0.29970222692398013         Other    NaN            NaN         NaN   
0.3676495157054054          Durbl    NaN            NaN         NaN   
0.37039809983930183           NaN    NaN            NaN       NoDur   
0.4458548795016067            NaN    NaN            NaN       Utils   
0.5018014853619303            NaN    NaN            NaN       Enrgy   
0.5640

In [26]:
import seaborn as sns
plt.style.use('seaborn')
plt.figure( figsize = (20,10) )
x  = Performance_metrics.unstack().reset_index(level =1).reset_index().rename( columns = {"index":"Ratio", 0:"Value"})
testing = sns.catplot(x="Ratio", y="Value", hue="Sector",data=x, kind="bar", height=10, aspect=2.0,legend=False)
plt.xticks(fontsize=16)
plt.yticks(fontsize=16)
plt.legend(fontsize=16)
plt.xlabel( "Ratios",fontsize=16 )
plt.ylabel( "Values",fontsize=16 )
plt.title("Factors comparision across different sectors",fontsize=16)

ValueError: Neither the `x` nor `y` variable appears to be numeric.

<Figure size 1440x720 with 0 Axes>

### Significance of different Ratios:
> 1. Treynor Ratio  : 
    >> It's equal to the risk premmium divided by beta  (systematic risk). It is amount of return expected for a given amount of  systematci risk taken.
    >> This ratio is more appropriate for comparing well diversified portfolios.
    >> Treynor measure considered as more forward looking measure.
    >> As per graph "Utils" has hightest Treynor ratio, hence it gives maximum amount of return for given value of risk 

> 2. Sharpe Ratio:
    >> Sharpe ratio is equal to the risk premium  divided by the standard deviation / total risk ( Systematic + unsystematic). It's a total expected return for total amount of risk taken
    >> it is better for comparing historical performance.
    >> NoDur has highest Sharpe Ratio compare to other sectors.

> 3. Sortinio Ratio
   >> The Sortino ratio is a variation of the Sharpe ratio that differentiates harmful volatility from total overall volatility by using the asset's standard deviation of negative portfolio returns—downside deviation—instead of the total standard deviation of portfolio returns. The Sortino ratio takes an asset or portfolio's return and subtracts the risk-free rate, and then divides that amount by the asset's downside deviation. 

> 4. Alpha
  >> Excess return over the return predeicted by the CAPM
  >> Most appropriate for comparing portfolios have same Beta 
  >> energy sector has hightest alpha in compare to other sectors

### Efficient Frontier 

In [ ]:
mean_returns = data_indus_Portfolio.mean()
mean_returns_array = np.array(mean_returns)
mean_returns_array.transpose()

In [ ]:
cov_matrix = data_indus_Portfolio.cov()
cov_matrix

In [ ]:
weights = []
mean_return = []
variance = []
for i in np.arange(1,1000000,1):
    rand_Num  = (np.array(np.random.uniform(1,0,10)))
    x = rand_Num /np.sum(rand_Num)
    weights.append(x)
    mean_return.append(x.dot(mean_returns_array.T))
    variance.append((x) @(cov_matrix) @ x.T)

In [ ]:
df_3 = pd.DataFrame( [ mean_return, np.sqrt(variance)])
df_3 = df_3.T
df_3.rename ( columns = {0:"Mean",1:"St_dev"},inplace = True)

In [ ]:
import seaborn as sns
plt.style.use('seaborn')
plt.figure( figsize =  (30,20) )
plt.scatter( x='St_dev', y='Mean',data=df_3,cmap='black', edgecolors='red',s=500,alpha = 0.75)
plt.xticks(fontsize=16)
plt.xlabel("Standard-Deviation",fontsize=30)
plt.ylabel("Expected Return",fontsize=30)
plt.xticks(fontsize=40)
plt.yticks(fontsize=40)
plt.title("Efficient Fontier ",fontsize=30)

### Efficient Frontier with weight 1/w 

In [ ]:
weights = []
mean_returns = []
var = []
for i in np.arange(1,1000000,1):
    rand_Num  = 1/(np.array(np.random.uniform(1,0,10)))
    x = rand_Num /np.sum(rand_Num)
    weights.append(x)
    mean_returns.append(x.dot(mean_returns_array.T))
    var.append((x) @(cov_matrix) @ x.T)

df_efficient_frontier = pd.DataFrame( [ mean_returns, var])
df_efficient_frontier = df_efficient_frontier.T
df_efficient_frontier.rename ( columns = {0:"Mean",1:"Variance"},inplace = True)
df_efficient_frontier["St_dev"] = np.sqrt(df_efficient_frontier["Variance"])

In [ ]:
plt.style.use('seaborn')
plt.figure( figsize =  (30,20) )
plt.scatter( x='St_dev', y='Mean',data=df_efficient_frontier,cmap='black', edgecolors='red',s=500,alpha = 0.25)
plt.xticks(fontsize=16)
plt.xlabel("Standard-Deviation",fontsize=30)
plt.ylabel("Expected Return",fontsize=30)
plt.xticks(fontsize=40)
plt.yticks(fontsize=40)
plt.title("Factors comparision across different sectors",fontsize=30)